In [ ]:
!apt-get update && apt-get install -y graphviz
!pip install expecttest pydot

In [ ]:
import torch
import torch.fx
import torch.nn as nn
print(torch.__version__)

In [ ]:
import tensorrt
print(tensorrt.__version__)

In [ ]:
import torch_tensorrt
print(torch_tensorrt.__version__)

In [ ]:
from torch_tensorrt.fx.tracer.acc_tracer import acc_normalizer, acc_ops, acc_shape_prop, acc_utils  # noqa: F401
from torch.fx.experimental.normalize import NormalizeArgs

import torch_tensorrt
from torch_tensorrt.fx.utils import LowerPrecision
import torch_tensorrt.fx.tracer.acc_tracer.acc_tracer as acc_tracer
from torch_tensorrt.fx import InputTensorSpec, TRTInterpreter, TRTModule
from torch_tensorrt.fx.tools.trt_splitter import TRTSplitter, TRTSplitterSetting

In [ ]:
from torch.fx.node import Argument, Target
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union
from torch_tensorrt.fx.converters.converter_utils import SourceIR
from torch_tensorrt.fx.converter_registry import tensorrt_converter
from torch_tensorrt.fx.tracer.acc_tracer.acc_op_properties import AccOpProperty, register_acc_op_properties
from torch_tensorrt.fx.tracer.acc_tracer.acc_normalizer import (
    register_acc_op,
    register_acc_op_mapping,
    register_custom_acc_mapper_fn,
)
from torch_tensorrt.fx.types import (
    TRTNetwork,
    TRTTensor,
)
from torch_tensorrt.fx.converters.impl import activation

In [ ]:
import tensorrt as trt
TRT_LOGGER = trt.Logger(trt.Logger.ERROR)
trt.init_libnvinfer_plugins(TRT_LOGGER, "")
print(f"Register libnvinfer plugins")
registry = trt.get_plugin_registry()
print(f"Registry: {registry}")
for plugin in registry.plugin_creator_list:
    print(plugin.name)

In [ ]:
from torch_tensorrt.fx.converter_registry import CONVERTERS
from torch_tensorrt.fx.tracer.acc_tracer.acc_normalizer import _acc_ops, _normalization_dict
from torch_tensorrt.fx.tracer.acc_tracer import acc_ops

print(">>" * 40)
print("Converters")
print(">>" * 40)
for op in CONVERTERS:
    print(op)
    
print(">>" * 40)
print("acc_ops")
print(">>" * 40)
for op in _acc_ops:
    print(op)
    
print(">>" * 40)
print("_normalization_dict")
print(">>" * 40)
for op in _normalization_dict:
    print(op)

In [ ]:
# for op in list(CONVERTERS.keys()):
#     if op == acc_ops.gelu:
#         CONVERTERS.pop(op)
#         print(f"removed converter {op}")

# for op in list(_acc_ops):
#     if op.__name__ == "gelu":
#         _acc_ops.remove(op)
#         print(f"removed acc_op: {op}")
        
# for (op, target) in list(_normalization_dict.keys()):
#     if "gelu" in str(target) or "GELU" in str(target):
#         _normalization_dict.pop((op, target))
#         print(f"removed normalization_dict op: {op}")

In [ ]:
import torch
import torch.nn as nn
import torch_tensorrt as torchtrt

# create a simple norm layer.
# This norm layer uses NormalizePlugin from Torch-TensorRT

class Norm(torch.nn.Module):
    def __init__(self, C: int):
        super(Norm, self).__init__()
        self.gn = nn.GroupNorm(C // 2, C)

    def forward(self, x):
        return self.gn(x)
#         num_groups = 2
#         return torch.nn.functional.group_norm(
#             x, num_groups, eps=1e-05
#         )

In [ ]:
C = 6
norm_model = Norm(C).eval().cuda()
# with torch.inference_mode():
#     norm_ts_module = torch.jit.script(norm_model)

In [ ]:
norm_ts_module.graph

In [ ]:
shape = [1, C, 64, 64]
x = torch.rand(*shape, dtype=torch.float32, device="cuda")
with torch.inference_mode():
    trt_traced = acc_tracer.trace(norm_model, [x])
print(trt_traced)

In [ ]:
# compile_settings = {
#     "inputs": [torchtrt.Input(shape, dtype=torch.float32)],
#     "enabled_precisions": {torch.float32},
# }

# norm_trt_ts = torchtrt.compile(norm_ts_module, **compile_settings)
# torch.jit.save(norm_trt_ts, "norm_trt_ts.pt")
# print("Generated Torchscript-TRT GroupNorm model.")

In [ ]:
# print(norm_trt_ts.graph)

In [ ]:
import torch
from typing import Sequence
from torch_tensorrt.dynamo.backend._settings import CompilationSettings
from torch_tensorrt.dynamo.backend.lowering._partition import (
    partition,
    get_submod_inputs,
)
from torch_tensorrt.dynamo.backend.lowering._partition import logger as plogger
from torch_tensorrt.dynamo.backend.conversion import convert_module


# def _compile_module(
#     gm: torch.fx.GraphModule,
#     sample_inputs: Sequence[torch.Tensor],
#     settings: CompilationSettings = CompilationSettings(),
# ) -> torch.fx.GraphModule:
#     """Compile a traced FX module

#     Includes: Partitioning + Conversion Phases

#     Args:
#         module: FX GraphModule to convert
#         inputs: Inputs to the module
#         settings: Compilation settings
#     Returns:
#         Compiled FX GraphModule
#     """
#     # Partition module into components that can be TRT-accelerated
#     level = plogger.level
#     plogger.setLevel(logging.DEBUG)
#     partitioned_module = partition(
#         gm,
#         verbose=settings.debug,
#         min_block_size=settings.min_block_size,
#         torch_executed_ops=settings.torch_executed_ops,
#     )
#     plogger.setLevel(level)

#     # Iterate over all components that can be accelerated
#     # Generate the corresponding TRT Module for those
#     for name, _ in partitioned_module.named_children():
#         if "_run_on_acc" not in name:
#             continue
#         submodule = getattr(partitioned_module, name)

#         # Get submodule inputs
#         submodule_inputs = get_submod_inputs(
#             partitioned_module, submodule, sample_inputs
#         )

#         # Create TRT Module from submodule
#         trt_mod = convert_module(
#             submodule,
#             submodule_inputs,
#             settings=settings,
#         )

#         # Replace FX Module with TRT Module
#         setattr(partitioned_module, name, trt_mod)

#     return partitioned_module

In [ ]:
# from torch_tensorrt.dynamo.backend.lowering._partition import (
#     partition,
#     get_submod_inputs,
# )
# from torch_tensorrt.dynamo.backend.lowering._partition import logger as plogger

# # with torch.inference_mode():
# #     splitter = TRTSplitter(trt_traced, [x])
# #     # splitter.node_support_preview(dump_graph=False)
# import logging
# logger = logging.getLogger(__name__)


# level = plogger.level
# plogger.setLevel(logging.DEBUG)
# partitioned_module = partition(
#     trt_traced,
#     verbose=True,
#     min_block_size=0,
#     torch_executed_ops=[],
# )
# plogger.setLevel(level)


# # Since the model is splitted into three segments. We need to lower each TRT eligible segment.
# # If we know the model can be fully lowered, we can skip the splitter part.
# inputs = [x]
# for name, _ in partitioned_module.named_children():
#     print(f"Splitting {name}")
#     if "_run_on_acc" in name:
#         submodule = getattr(partitioned_module, name)

#         # Get submodule inputs for fx2trt
#         submodule_inputs = get_submod_inputs(partitioned_module, submodule, inputs)

#         # Create TRT Module from submodule
#         trt_mod = convert_module(
#             submodule,
#             submodule_inputs,
#             settings=settings,
#         )
        
#         setattr(partitioned_module, name, trt_mod)

In [ ]:
# del model
# del pipe
# del traced_unet

with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
import torch
import torch.nn as nn
import torch_tensorrt as torchtrt
from torch_tensorrt.fx.tools.trt_splitter import TRTSplitter, TRTSplitterSetting

import nos.compilers.trt as nostrt
import nos.compilers.trt.ops
from nos.compilers.trt.ops import group_norm as gn
import torch_tensorrt.fx.tracer.acc_tracer.acc_tracer as acc_tracer
from torch_tensorrt.fx.utils import LowerPrecision
from torch_tensorrt.dynamo.backend._defaults import (
    PRECISION,
    DEBUG,
    MAX_WORKSPACE_SIZE,
    MIN_BLOCK_SIZE,
    PASS_THROUGH_BUILD_FAILURES,
)

In [ ]:
from torch import fx
from transformers.utils.fx import HFTracer, get_concrete_args
from transformers import CLIPModel

import torch_tensorrt.fx.converter_registry as registry
registry.CONVERTERS.pop(acc_ops.expand)

dtype = torch.float32
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").cuda().eval()
model = model.vision_model
if dtype == torch.float32: # CLIP is already in half-mode
    model = model.float()
    
shape = [1, 3, 224, 224]
x = torch.randn(shape, dtype=dtype, device='cuda')
args = {
    "pixel_values": x
}
concrete_args = {
    "output_attentions": None,
    "output_hidden_states": None,
    "return_dict": True
}
inputs = list(args.values())

tracer = HFTracer()
traced_graph = tracer.trace(model, concrete_args=concrete_args, dummy_inputs=args)
traced_model = torch.fx.GraphModule(model, traced_graph)

In [ ]:
from diffusers import DiffusionPipeline
from diffusers import DDIMScheduler
from diffusers.pipelines.stable_diffusion import StableDiffusionPipeline
from diffusers.models.unet_2d_condition import UNet2DConditionOutput

dtype = torch.float32
# Use the DDIMScheduler scheduler here instead
scheduler = DDIMScheduler.from_pretrained("stabilityai/stable-diffusion-2-1",
                                            subfolder="scheduler")
# pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=dtype, scheduler=scheduler)
pipe = pipe.to("cuda")

In [ ]:
from torch import fx
from transformers.utils.fx import HFTracer

model = pipe.vae

# https://github.com/huggingface/diffusers/blob/666743302ff5bd1e02c204b81a80e566648d60de/examples/community/stable_diffusion_tensorrt_txt2img.py#L493
H, W = 512, 512 
B = 1
x = torch.rand(B, 3, H // 8, W // 8, dtype=dtype, device="cuda")
args = {
    'sample': x,
}
concrete_args = {
    "sample_posterior": False,
    "return_dict": True,
    "generator": None,
}
inputs = list(args.values())

tracer = HFTracer()
with torch.inference_mode():
    traced_graph = tracer.trace(model, concrete_args=concrete_args, dummy_inputs=args)
    traced_model = torch.fx.GraphModule(model, traced_graph)
# print(traced_graph.print_tabular())

traced_model.config = model.config
# The model class must be stored as an attribute to allow model deserialization, which uses trace, and thus
# _generate_dummy_input, where the model class is needed.
traced_model.class_for_deserialization = model.__class__
traced_model.device = model.device

In [ ]:
from torch import fx
from transformers.utils.fx import HFTracer

model = pipe.unet

# https://github.com/huggingface/diffusers/blob/666743302ff5bd1e02c204b81a80e566648d60de/examples/community/stable_diffusion_tensorrt_txt2img.py#L493
H, W = 512, 512
B = 1
D = 1024  # D = (v1-5: 768, v2-1: 1024)
unet_dim = 4
text_maxlen = 77

device = "cuda"
sample = torch.rand(2 * B, unet_dim, 96, 96, dtype=dtype, device=device)
timestep = torch.tensor([1, ], dtype=torch.int32, device=device)
encoder_hidden_states = torch.rand((2 * B, text_maxlen, D), dtype=dtype, device=device)  # 2B, 77, 1024

args = {
    'sample': sample,
    'timestep' : timestep,
    'encoder_hidden_states': encoder_hidden_states,
}
concrete_args = {
    'class_labels': None,
    'timestep_cond' : None,
    'attention_mask': None,
    'cross_attention_kwargs': None,
    'added_cond_kwargs': None,
    'down_block_additional_residuals': None,
    'mid_block_additional_residual': None,
    'encoder_attention_mask':None,
    'return_dict': True,
    'hidden_states': None,
    'cond_proj_dim' : 1,
    'condition': None,
    'use_ada_layer_norm': True,
    'input_ndim': 4,
}
inputs = list(args.values())

tracer = HFTracer()
# with torch.inference_mode():
traced_graph = tracer.trace(model, concrete_args={**args, **concrete_args})
traced_model = torch.fx.GraphModule(model, traced_graph)
# print(traced_graph.print_tabular())

traced_model.config = model.config
# The model class must be stored as an attribute to allow model deserialization, which uses trace, and thus
# _generate_dummy_input, where the model class is needed.
traced_model.class_for_deserialization = model.__class__
traced_model.device = model.device

In [ ]:
# model(*inputs)

In [ ]:
# import torch.nn as nn

# class TracedModelWithNoInputs(nn.Module):
#     def __init__(self, traced_model):
#         super().__init__()
#         self._model = traced_model
#         self._input_idx2name = {}
#         for key in dir(self._model):
#             if key.startswith("_tensor_constant"):
#                 self._input_idx2name[len(self._input_idx2name)] = key
#         print(f"Found inputs: {len(self._input_idx2name)}")

#         self.config = None
#         self.class_for_deserialization = None
#         self.device = None
        
#     def forward(self, sample, timestep, encoder_hidden_states, class_labels=None, cross_attention_kwargs=None, return_dict=True):
#         args = (sample, timestep, encoder_hidden_states)
#         idx = 0
#         for arg in args:
#             key = self._input_idx2name[idx]
#             setattr(self._model, key, arg)
#             print(f"setting {key}: {arg.shape}, old: {getattr(self._model, key).shape}")
#             idx += 1
#         # for _key, value in kwargs.items():
#         #     print(f"kwarg: {_key}")
#         #     if idx not in self._input_idx2name:
#         #         break
#         #     key = self._input_idx2name[idx]
#         #     setattr(self._model, key, value)
#         #     print(f"setting {key}: {value.shape}")
#         #     idx += 1
#         return UNet2DConditionOutput(self._model())
        
# traced_unet = TracedModelWithNoInputs(traced_model)
# traced_unet.config = pipe.unet.config
# # The model class must be stored as an attribute to allow model deserialization, which uses trace, and thus
# # _generate_dummy_input, where the model class is needed.
# traced_unet.class_for_deserialization = pipe.unet.__class__
# traced_unet.device = pipe.unet.device
# pipe.unet = traced_unet
# # with torch.inference_mode():
# #     result = traced_unet(*inputs)
# # result

In [ ]:
images = pipe(prompt=["fox jumped over dog"], num_inference_steps=5, num_images_per_prompt=1).images

In [ ]:
images[0]

In [ ]:
import logging
from typing import Sequence
from functools import partial

import torch
import torch._dynamo as td
from torch_tensorrt.dynamo.backend.lowering._decompositions import (
    get_decompositions,
)
from torch_tensorrt.dynamo.backend.lowering._partition import (
    partition,
    get_submod_inputs,
)
from torch_tensorrt.dynamo.backend._settings import CompilationSettings
from torch_tensorrt.dynamo.backend.utils import prepare_inputs, prepare_device
from torch_tensorrt.dynamo.backend.backends import aot_torch_tensorrt_aten_backend
from torch_tensorrt.dynamo.backend._defaults import (
    PRECISION,
    DEBUG,
    MAX_WORKSPACE_SIZE,
    MIN_BLOCK_SIZE,
    PASS_THROUGH_BUILD_FAILURES,
)
from torch._dynamo.backends.common import fake_tensor_unsupported
from torch._functorch.aot_autograd import aot_module_simplified, make_boxed_compiler
from torch_tensorrt.dynamo.backend.lowering._partition import logger as plogger
from torch_tensorrt.dynamo.backend.conversion import convert_module

from nos.compilers.trt.ops import group_norm as gn
# from nos.compilers.trt.backends import nos_tensorrt_backend


from torch._functorch.aot_autograd import aot_module_simplified, make_boxed_compiler

logger = logging.getLogger(__name__)


def _compile_module(
    gm: torch.fx.GraphModule,
    sample_inputs: Sequence[torch.Tensor],
    settings: CompilationSettings = CompilationSettings(),
) -> torch.fx.GraphModule:
    """Compile a traced FX module

    Includes: Partitioning + Conversion Phases

    Args:
        module: FX GraphModule to convert
        inputs: Inputs to the module
        settings: Compilation settings
    Returns:
        Compiled FX GraphModule
    """
    # Partition module into components that can be TRT-accelerated
    print(f"NOS compile:")
    level = plogger.level
    plogger.setLevel(logging.DEBUG)
    partitioned_module = partition(
        gm,
        verbose=settings.debug,
        min_block_size=settings.min_block_size,
        torch_executed_ops=settings.torch_executed_ops,
    )
    plogger.setLevel(level)

    # Iterate over all components that can be accelerated
    # Generate the corresponding TRT Module for those
    for name, _ in partitioned_module.named_children():
        # TOFIX (spillai): Investigate why this is needed here, 
        # but not in torch dynamo.
        if "_run_on_acc" not in name:
            print(f"Skipping partition: {name}")
            continue
        submodule = getattr(partitioned_module, name)

        # Get submodule inputs
        submodule_inputs = get_submod_inputs(
            partitioned_module, submodule, sample_inputs
        )

        # Create TRT Module from submodule
        trt_mod = convert_module(
            submodule,
            submodule_inputs,
            settings=settings,
        )

        # Replace FX Module with TRT Module
        setattr(partitioned_module, name, trt_mod)

    return partitioned_module

@fake_tensor_unsupported
def _pretraced_backend(
    gm: torch.fx.GraphModule,
    sample_inputs: Sequence[torch.Tensor],
    settings: CompilationSettings = CompilationSettings(),
):
    """Helper function to manage translation of traced FX module to TRT engines

    Args:
        module: FX GraphModule to convert
        inputs: Inputs to the module
        settings: Compilation settings
    Returns:
        Compiled FX GraphModule
    """
    try:
        trt_compiled = _compile_module(
            gm,
            sample_inputs,
            settings=settings,
        )
        return trt_compiled
    except:
        logger.error(
            "FX2TRT conversion failed on the subgraph. See trace above. "
            + "Returning GraphModule forward instead.",
            exc_info=True,
        )

        if not settings.pass_through_build_failures:
            return gm.forward
        else:
            raise AssertionError(
                "Halting compilation on build failure since "
                + "pass_through_build_failures was specified as True. "
                + "To return the default Torch implementation and avoid "
                + "halting compilation on engine build failures, "
                + "specify pass_through_build_failures=False."
            )



@fake_tensor_unsupported
def nos_tensorrt_backend(
    gm: torch.fx.GraphModule,
    sample_inputs: Sequence[torch.Tensor],
    settings: CompilationSettings = CompilationSettings(),
):
    custom_backend = partial(
        _pretraced_backend,
        settings=settings,
    )

    print("nos_trt_backend() called with FX graph:")
    gm.graph.print_tabular()

    # print("Acc Tracer")
    # with torch.inference_mode():
    #     gm = acc_tracer.trace(
    #         gm, sample_inputs, 
    #     )

    # Invoke AOTAutograd to translate operators to aten
    return _pretraced_backend(gm, sample_inputs, settings=settings)
    
    # return aot_module_simplified(
    #     gm,
    #     sample_inputs,
    #     fw_compiler=make_boxed_compiler(custom_backend),
    #     decompositions=get_decompositions(),
    # )


def create_backend(
    precision: LowerPrecision = PRECISION,
    debug: bool = DEBUG,
    workspace_size: int = MAX_WORKSPACE_SIZE,
    min_block_size: int = MIN_BLOCK_SIZE,
    torch_executed_ops: Sequence[str] = set(),
    pass_through_build_failures: bool = PASS_THROUGH_BUILD_FAILURES,
    **kwargs,
):
    """Create torch.compile backend given specified arguments

    Args:
        precision: Model Layer precision
        debug: Whether to print out verbose debugging information
        workspace_size: Workspace TRT is allowed to use for the module (0 is default)
        min_block_size: Minimum number of operators per TRT-Engine Block
        torch_executed_ops: Sequence of operations to run in Torch, regardless of converter coverage
        pass_through_build_failures: Whether to fail on TRT engine build errors (True) or not (False)
    Returns:
        Backend for torch.compile
    """
    if debug:
        logger.setLevel(logging.DEBUG)

    settings = CompilationSettings(
        debug=debug,
        precision=precision,
        workspace_size=workspace_size,
        min_block_size=min_block_size,
        torch_executed_ops=torch_executed_ops,
        pass_through_build_failures=pass_through_build_failures,
    )

    return partial(
        nos_tensorrt_backend,
        settings=settings,
    )

In [ ]:
import torch
import logging
import collections.abc
import torch_tensorrt
from functools import partial

from typing import Any, Sequence
from torch_tensorrt import EngineCapability, Device
from torch_tensorrt.fx.utils import LowerPrecision


torch._dynamo.config.log_level = logging.INFO
torch._dynamo.config.verbose = True
torch._dynamo.config.suppress_errors = True

debug = True
custom_backend = create_backend(
    precision=LowerPrecision.FP32 if dtype == torch.float32 else LowerPrecision.FP16,
    debug=debug,
    workspace_size=MAX_WORKSPACE_SIZE,
    min_block_size=0, # MIN_BLOCK_SIZE,
    torch_executed_ops=[],
)

torch._dynamo.eval_frame.remove_from_cache(model)
with torch.inference_mode():
    trt_model = torch_tensorrt.compile(traced_unet, backend=custom_backend)

with torch.inference_mode():
    trt_model(x)

# Clean up model env
# torch._dynamo.reset()

# with torch.no_grad():
#     torch.cuda.empty_cache()

In [ ]:
trt_model = torch_tensorrt.fx.compile(traced_model, 
                                    inputs,
                                    min_acc_module_size=0,
                                    max_batch_size=2048,
                                    max_workspace_size=33554432,
                                    lower_precision=LowerPrecision.FP32 if dtype == torch.float32 else LowerPrecision.FP16,
                                    verbose_log=True,
                                    timing_cache_prefix='',
                                    save_timing_cache=False,
                                    cuda_graph_batch_size=-1,
                                    is_aten=False,
                                    use_experimental_fx_rt=False,
                                    explicit_batch_dimension=True, 
                                    dynamic_batch=False
)

In [ ]:
pipe.vae = trt_model

In [ ]:
images = pipe(prompt=["fox jumped over dog"], num_inference_steps=200, num_images_per_prompt=1).images

In [ ]:
filename = "norm_trt_engine.pt"
torch.save(trt_model, filename)
trt_model = torch.load(filename)

In [ ]:
with torch.inference_mode():
    result = model(x)
# result

In [ ]:
with torch.inference_mode():
    result = trt_model(x)
# result

In [ ]:
%%timeit -n 10 -r 3
with torch.inference_mode():
    model(x)

In [ ]:
%%timeit -n 10 -r 3
with torch.inference_mode():
    trt_model(x)

In [ ]:
trt_mod(x)

In [ ]:
%%